# Temporal feature selection with shap values


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pip install --upgrade 'lightgbm>=3.0.0'
!pip install "pycaret"

In [ ]:
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt
import pdb
import warnings
import seaborn as sns
import shap

from sklearn import metrics
from pycaret.classification import *
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.metrics import roc_auc_score
from sklearn.metrics import log_loss, f1_score, accuracy_score
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.feature_extraction import stop_words
from sklearn.feature_extraction.text import TfidfVectorizer
from lightgbm import LGBMClassifier

plt.style.use('fivethirtyeight')
plt.rcParams['figure.figsize'] = (12, 4)
warnings.filterwarnings('ignore')

In [ ]:
STOP_WORDS = ['de', 'a', 'o', 'que', 'e', 'do', 'da', 'em', 'um', 'para', 'é', 'com', 'não', 'uma', 'os', 'no', 'se', 'na', 'por', 'mais', 'as', 'dos', 'como', 'mas', 'foi', 'ao', 'ele', 'das', 'tem', 'à', 'seu', 'sua', 'ou', 'ser', 'quando', 'muito', 'há', 'nos', 'já', 'está', 'eu', 'também', 'só', 'pelo', 'pela', 'até', 'isso', 'ela', 'entre', 'era', 'depois', 'sem', 'mesmo', 'aos', 'ter', 'seus', 'quem', 'nas', 'me', 'esse', 'eles', 'estão', 'você', 'tinha', 'foram', 'essa', 'num', 'nem', 'suas', 'meu', 'às', 'minha', 'têm', 'numa', 'pelos', 'elas', 'havia', 'seja', 'qual', 'será', 'nós', 'tenho', 'lhe', 'deles', 'essas', 'esses', 'pelas', 'este', 'fosse', 'dele', 'tu', 'te', 'vocês', 'vos', 'lhes', 'meus', 'minhas', 'teu', 'tua', 'teus', 'tuas', 'nosso', 'nossa', 'nossos', 'nossas', 'dela', 'delas', 'esta', 'estes', 'estas', 'aquele', 'aquela', 'aqueles', 'aquelas', 'isto', 'aquilo', 'estou', 'está', 'estamos', 'estão', 'estive', 'esteve', 'estivemos', 'estiveram', 'estava', 'estávamos', 'estavam', 'estivera', 'estivéramos', 'esteja', 'estejamos', 'estejam', 'estivesse', 'estivéssemos', 'estivessem', 'estiver', 'estivermos', 'estiverem', 'hei', 'há', 'havemos', 'hão', 'houve', 'houvemos', 'houveram', 'houvera', 'houvéramos', 'haja', 'hajamos', 'hajam', 'houvesse', 'houvéssemos', 'houvessem', 'houver', 'houvermos', 'houverem', 'houverei', 'houverá', 'houveremos', 'houverão', 'houveria', 'houveríamos', 'houveriam', 'sou', 'somos', 'são', 'era', 'éramos', 'eram', 'fui', 'foi', 'fomos', 'foram', 'fora', 'fôramos', 'seja', 'sejamos', 'sejam', 'fosse', 'fôssemos', 'fossem', 'for', 'formos', 'forem', 'serei', 'será', 'seremos', 'serão', 'seria', 'seríamos', 'seriam', 'tenho', 'tem', 'temos', 'tém', 'tinha', 'tínhamos', 'tinham', 'tive', 'teve', 'tivemos', 'tiveram', 'tivera', 'tivéramos', 'tenha', 'tenhamos', 'tenham', 'tivesse', 'tivéssemos', 'tivessem', 'tiver', 'tivermos', 'tiverem', 'terei', 'terá', 'teremos', 'terão', 'teria', 'teríamos', 'teriam']
CLUB_WORDS = ["verdão", "tricolor", "fla", "timão", "rubro", 
              "negro", "flamengo", "paulo", "palmeirense", "paulista", 
              "inter", "colorado", "internacional", "colorados", "colorada"]
STOP_WORDS += CLUB_WORDS
MAX_VOCABULARY = 1000
positive_case = "flamengo"
train_end_date = "2018-01"
holdout_end_date = "2020-10"
target = "target"
time_column = "year-month"

In [ ]:
def fix_small_data_period(data):
    data = data[(data["year-month"] < "2015-12") | (data["year-month"] > "2016-08")]   
    return data 

def exclude_periods_without_positive_case(data, positive_case, period_column, threshold=20):
    df = data.groupby(period_column)["club"].apply(lambda x: np.sum(x == positive_case))
    df = df[df > threshold]
    return data[data[period_column].isin(df.index)]
    
def clean_club_name_from_article(data):
    data["text"] = data.apply(lambda x: x["text"].lower().replace(x["club"].replace("-", " "), ""), axis=1)
    return data

def exclude_numbers(data):
    data["text"] = data["text"].apply(lambda x: ''.join([i for i in x.lower() if not i.isdigit()]))
    return data

def drop_multiple_teams_news(data):
    multiple_team_news = data.groupby("link", as_index=False)["club"].count()
    multiple_team_news = multiple_team_news[multiple_team_news["club"] > 1]["link"]
    return data[~data["link"].isin(multiple_team_news)]

In [ ]:
data = pd.read_csv("/kaggle/input/ge-soccer-clubs-news/ge_news.csv", index_col=False)

In [ ]:
data.shape

In [ ]:
data = clean_club_name_from_article(data)
data = exclude_numbers(data)
data = drop_multiple_teams_news(data)

In [ ]:
data.loc[:, "year"] = data["date"].apply(lambda x: x.split("/")[-1])
data.loc[:, "month"] = data["date"].apply(lambda x: x.split("/")[1])
data.loc[:, "date"] = pd.to_datetime(data["date"])

In [ ]:
### Monthly context
data["year-month"] = data["year"] + "-" + data["month"]

In [ ]:
data["target"] = data["club"].apply(lambda x: 1 if x == positive_case else 0)

print("The fraction of positive cases is {:.2f}".format(data["target"].mean()))

In [ ]:
data = exclude_periods_without_positive_case(data, positive_case, "year-month")

In [ ]:
data.shape

In [ ]:
in_time = data[data["year-month"] < train_end_date]

train, test = train_test_split(in_time, 
                               test_size=0.2, 
                               random_state=42)

out_of_time = data[(data["year-month"] >= train_end_date) & (data["year-month"] <= holdout_end_date)]

In [ ]:
print("Dataset shapes:")
print("Train: {}".format(train.shape))
print("Test: {}".format(test.shape))
print("Out of time: {}".format(out_of_time.shape))

## Calculating Features instability

In [ ]:
vectorizer = TfidfVectorizer(max_features=MAX_VOCABULARY,
                                 stop_words=STOP_WORDS,
                                 binary=True,
                                 use_idf=True)

vectorizer = vectorizer.fit(train["text"])

train_vectors = pd.DataFrame(vectorizer.transform(train["text"]).toarray(), columns=vectorizer.vocabulary_)
test_vectors = pd.DataFrame(vectorizer.transform(test["text"]).toarray(), columns=vectorizer.vocabulary_)
oot_vectors = pd.DataFrame(vectorizer.transform(out_of_time["text"]).toarray(), columns=vectorizer.vocabulary_)

In [ ]:
model = LGBMClassifier()
model.fit(train_vectors, train["target"].values)

In [ ]:
experiment_dict = {}
for period in in_time[time_column].unique():
    print(period)
    in_time_period = in_time[in_time[time_column] == period]
    test_period = test[test[time_column] == period]

    test_vectors_ = pd.DataFrame(vectorizer.transform(test_period["text"]).toarray(), columns=vectorizer.vocabulary_)
    predictions = model.predict(test_vectors_, pred_contrib=True)
    experiment_dict[period] = predictions

In [ ]:
importance = []
for key, value in experiment_dict.items():
    shap_contrib = np.abs(value[:, :-1]).mean(0)
    df = pd.DataFrame(shap_contrib.reshape(1, MAX_VOCABULARY), columns=vectorizer.vocabulary_)
    df["period"] = key
    importance.append(df)

importance = pd.concat(importance)

In [ ]:
average = importance.mean()

In [ ]:
average.sort_values()

In [ ]:
deviation = importance.std() / average

In [ ]:
deviation.fillna(0, inplace=True)
deviation

In [ ]:
median_average_contrib = average.median()
median_std_contrib = deviation.median()

median_average_contrib = np.percentile(average, 50)
median_std_contrib = np.percentile(deviation, 80)

In [ ]:
plt.scatter(average, deviation)

xmin, xmax, ymin, ymax = plt.axis()
plt.hlines(median_std_contrib, xmin, xmax, linestyle="dotted", color="red")
plt.vlines(median_average_contrib, ymin, ymax, linestyle="dotted", color="red")
plt.legend(bbox_to_anchor=(1.05, 1.0))
plt.title("Contribution x Instability")
plt.ylabel("Instability")
plt.xlabel("Contribution")
plt.show()

In [ ]:
aggregate_importance = pd.DataFrame()
aggregate_importance["average"] = average.values
aggregate_importance["instability"] = deviation.values
aggregate_importance.index = average.index

aggregate_importance

In [ ]:
mask = (aggregate_importance["average"] >= median_average_contrib) & (aggregate_importance["instability"] <= median_std_contrib)
stable_features = aggregate_importance[mask].index

## Comparing a model using stable features Vs model using all features

### Benchmark (All features)

In [ ]:
train_vectors = pd.DataFrame(vectorizer.transform(train["text"]).toarray(), columns=vectorizer.vocabulary_)
test_vectors = pd.DataFrame(vectorizer.transform(test["text"]).toarray(), columns=vectorizer.vocabulary_)

In [ ]:
in_time_data = pd.DataFrame(vectorizer.transform(in_time["text"]).toarray(), columns=vectorizer.vocabulary_)
in_time_data[target] = in_time[target].values

In [ ]:
clf1 = setup(in_time_data, target=target,
                      session_id=123, 
                      log_experiment=True, 
                      experiment_name="exp1",
            silent=True)

lgbm = create_model("lightgbm")

tuned_lgbm = tune_model(lgbm,
                        fold=5,
                        n_iter=50,
                        optimize="AUC")

best = automl(optimize = 'AUC')

In [ ]:
model = best
model.fit(train_vectors, train["target"].values)
predictions = model.predict_proba(test_vectors)[:, 1]
test["benchmark"] = predictions
roc_auc_score(test["target"], predictions)

In [ ]:
predictions = model.predict_proba(oot_vectors)[:, 1]
out_of_time["benchmark"] = predictions
roc_auc_score(out_of_time["target"], predictions)

In [ ]:
out_of_time.groupby(time_column).apply(lambda x: roc_auc_score(x[target], x["benchmark"])).plot()
plt.title("AUC for the out of time set (unseen future data)")
plt.xlabel("Period (monthly)")
plt.ylabel("AUC")
plt.legend()

In [ ]:
out_of_time.groupby(time_column).apply(lambda x: roc_auc_score(x[target], x["benchmark"])).rolling(6).mean().plot()
plt.title("Moving average AUC with a 6 months window in the out of time set")
plt.ylabel("AUC")
plt.xlabel("Period (monthly)")
plt.show()

### Stable features model (challenger)

In [ ]:
clf2 = setup(in_time_data[list(stable_features) + [target]], target=target,
                      session_id=123, 
                      log_experiment=True, 
                      experiment_name="exp1",
                     silent=True)

lgbm = create_model("lightgbm")

tuned_lgbm = tune_model(lgbm,
                        fold=5,
                        n_iter=50,
                        optimize="AUC")

best = automl(optimize = 'AUC')
model = best

In [ ]:
model.fit(train_vectors[stable_features], train["target"].values)

In [ ]:
predictions = model.predict_proba(test_vectors[stable_features], pred_contrib=True)
explainer = shap.TreeExplainer(model)
shap_values = explainer.shap_values(test_vectors[stable_features])
shap.summary_plot(shap_values, test_vectors[stable_features], plot_type="bar")

In [ ]:
predictions = model.predict_proba(test_vectors[stable_features])[:, 1]
test["temporal_prediction"] = predictions
print(roc_auc_score(test["target"], predictions))
predictions = model.predict_proba(oot_vectors[stable_features])[:, 1]
out_of_time["temporal_prediction"] = predictions
print(roc_auc_score(out_of_time["target"], predictions))

In [ ]:
pd.concat([test, out_of_time]).groupby("year").apply(lambda x: roc_auc_score(x[target], x["benchmark"])).plot(label="All features (benchmark)")
pd.concat([test, out_of_time]).groupby("year").apply(lambda x: roc_auc_score(x[target], x["temporal_prediction"])).plot(label="Stable features(challenger)", 
                                                                                                                        color="green")
plt.legend()
plt.title("AUC")

In [ ]:
pd.concat([test, out_of_time]).groupby("year").apply(lambda x: log_loss(x[target], x["benchmark"])).plot(label="All features (benchmark)")
pd.concat([test, out_of_time]).groupby("year").apply(lambda x: log_loss(x[target], x["temporal_prediction"])).plot(label="Stable features(challenger)", color="green")
plt.legend()
plt.title("Log loss")

In [ ]:
pd.concat([test, out_of_time]).groupby(time_column).apply(lambda x: roc_auc_score(x[target], x["benchmark"])).plot(label="All features (benchmark)")
pd.concat([test, out_of_time]).groupby(time_column).apply(lambda x: roc_auc_score(x[target], x["temporal_prediction"])).plot(label="Stable features(challenger)")
plt.legend()
plt.title("AUC")

In [ ]:
fig, ax = plt.subplots(figsize=(12, 4))
pd.concat([test, out_of_time]).groupby(time_column).apply(lambda x: roc_auc_score(x[target], x["benchmark"])).rolling(6).mean().plot(label="All features (benchmark)")
pd.concat([test, out_of_time]).groupby(time_column).apply(lambda x: roc_auc_score(x[target], x["temporal_prediction"])).rolling(6).mean().plot(label="Stable features (challenger)", color="green")

xmin, xmax, ymin, ymax = plt.axis()
train_end_date_position = np.argmax(np.sort(pd.concat([test, out_of_time])[time_column].unique()) == train_end_date)
plt.vlines(train_end_date_position, ymin, ymax, linestyle="dotted", color="red", label="Out of time split")
plt.legend()
plt.title("Moving average AUC with a 6 months window for both test and out of time periods", pad=16)
plt.ylabel("AUC")
plt.xlabel("Period (monthly)")
plt.show()
